In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Muralikrishna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Muralikrishna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Muralikrishna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# lopading the data set
df = pd.read_csv('BBC_DATA.csv')

In [4]:
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [5]:
df.tail()

,ArticleId,Text,Category
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech
1489,538,santy worm makes unwelcome visit thousands of ...,tech


In [6]:
df.describe()

,ArticleId
count,1490.000000
mean,1119.696644
std,641.826283
min,2.000000
25%,565.250000
50%,1112.500000
75%,1680.750000
max,2224.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Muralikrishna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Muralikrishna\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
# tokenize with NLTK
def tokenize_text(text1):
    words = word_tokenize(text1)
    return words

sample_text = df['Text'].iloc[0]
tokenized_text = tokenize_text(sample_text)
print(tokenized_text)

['worldcom', 'ex-boss', 'launches', 'defence', 'lawyers', 'defending', 'former', 'worldcom', 'chief', 'bernie', 'ebbers', 'against', 'a', 'battery', 'of', 'fraud', 'charges', 'have', 'called', 'a', 'company', 'whistleblower', 'as', 'their', 'first', 'witness', '.', 'cynthia', 'cooper', 'worldcom', 's', 'ex-head', 'of', 'internal', 'accounting', 'alerted', 'directors', 'to', 'irregular', 'accounting', 'practices', 'at', 'the', 'us', 'telecoms', 'giant', 'in', '2002.', 'her', 'warnings', 'led', 'to', 'the', 'collapse', 'of', 'the', 'firm', 'following', 'the', 'discovery', 'of', 'an', '$', '11bn', '(', '£5.7bn', ')', 'accounting', 'fraud', '.', 'mr', 'ebbers', 'has', 'pleaded', 'not', 'guilty', 'to', 'charges', 'of', 'fraud', 'and', 'conspiracy', '.', 'prosecution', 'lawyers', 'have', 'argued', 'that', 'mr', 'ebbers', 'orchestrated', 'a', 'series', 'of', 'accounting', 'tricks', 'at', 'worldcom', 'ordering', 'employees', 'to', 'hide', 'expenses', 'and', 'inflate', 'revenues', 'to', 'meet',

In [11]:
#Stemming and Lemmatization with NLTK:

def stem_lemmatize_text(Text):
    words = word_tokenize(Text)
    words = [word for word in words if word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return lemmatized_words

stemmed_lemmatized_text = stem_lemmatize_text(sample_text)
print(stemmed_lemmatized_text)

['worldcom', 'ex-boss', 'launch', 'defenc', 'lawyer', 'defend', 'former', 'worldcom', 'chief', 'berni', 'ebber', 'batteri', 'fraud', 'charg', 'call', 'compani', 'whistleblow', 'first', 'wit', '.', 'cynthia', 'cooper', 'worldcom', 'ex-head', 'intern', 'account', 'alert', 'director', 'irregular', 'account', 'practic', 'u', 'telecom', 'giant', '2002.', 'warn', 'led', 'collaps', 'firm', 'follow', 'discoveri', '$', '11bn', '(', '£5.7bn', ')', 'account', 'fraud', '.', 'mr', 'ebber', 'plead', 'guilti', 'charg', 'fraud', 'conspiraci', '.', 'prosecut', 'lawyer', 'argu', 'mr', 'ebber', 'orchestr', 'seri', 'account', 'trick', 'worldcom', 'order', 'employe', 'hide', 'expens', 'inflat', 'revenu', 'meet', 'wall', 'street', 'earn', 'estim', '.', 'm', 'cooper', 'run', 'consult', 'busi', 'told', 'juri', 'new', 'york', 'wednesday', 'extern', 'auditor', 'arthur', 'andersen', 'approv', 'worldcom', 'account', 'earli', '2001', '2002.', 'said', 'andersen', 'given', 'green', 'light', 'procedur', 'practic', 'u

In [ ]:
#Named Entity Recognition with SpaCy:
from spacy import displacy
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

def named_entity_recognition(text1):
    doc = nlp(text1)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

sample_text = df['Text'].iloc[0]
named_entities = named_entity_recognition(sample_text)
print(named_entities)

doc = nlp(sample_text)
displacy.serve(doc, style="ent")

     -------------------------------------- 12.8/12.8 MB 937.4 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[('worldcom ex-boss', 'PERSON'), ('worldcom', 'ORG'), ('bernie', 'PERSON'), ('first', 'ORDINAL'), ('cynthia cooper  worldcom s ex-head', 'PERSON'), ('us', 'GPE'), ('2002', 'DATE'), ('5.7bn', 'MONEY'), ('worldcom', 'ORG'), ('new york', 'GPE'), ('wednesday', 'DATE'), ('arthur andersen', 'PERSON'), ('worldcom', 'ORG'), ('early 2001 and 2002', 'DATE'), ('worldcom', 'ORG'), ('cooper', 'PERSON'), ('worldcom financial', 'ORG'), ('scott sullivan', 'PERSON'), ('sullivan', 'PERSON'), ('2001', 'DATE'), ('85 years', 'DATE'), ('worldcom', 'ORG'), ('2004', 'DATE'), ('mci', 'ORG'), ('last week', 'DATE'), ('mci', 'ORG'), ('6.75bn', 'MONEY')]


C:\Users\Muralikrishna\anaconda3\lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



In [18]:
#pip install spacy
#python -m spacy download en_core_web_sm

In [14]:
#Word2Vec with gensim:
def word2vec_model(text1):
    sentences = [word_tokenize(sentence) for sentence in text1]
    model = Word2Vec(sentences, min_count=1)
    return model

model = word2vec_model(df['Text'])
sample_word = df['Text'].iloc[0].split()[0]
sample_word_vector = model.wv[sample_word]
print(sample_word_vector)

[-0.1539649   0.28397778 -0.08507434  0.07107157  0.25331858 -0.5554132
  0.07667944  0.4252339  -0.11645254 -0.10605089 -0.04155336 -0.05034972
 -0.00285643  0.11641409  0.35854957 -0.47424757  0.18794382 -0.34297553
  0.41346177 -0.16677886  0.25451028  0.05334672  0.41619518 -0.16157596
  0.2676314  -0.3405297  -0.5635787   0.36808547 -0.30493018  0.12471279
  0.5633669  -0.03471898  0.08114015 -0.5641541   0.04146161  0.3245079
  0.07348119 -0.2198489  -0.02277398 -0.31551462 -0.1728876  -0.24567193
 -0.1783751   0.09243151  0.3801851   0.2567218  -0.4632815   0.23587994
 -0.00516262  0.0110475   0.25436962 -0.39976844 -0.2834374   0.14808747
  0.02307659  0.33615726  0.3095228   0.19163743 -0.4184463  -0.01400392
  0.09631667  0.07427723 -0.34392467  0.06755113  0.03913922  0.323065
 -0.10172676  0.34213236 -0.27390984  0.3817969  -0.0569018   0.24556878
  0.52225316  0.17475472  0.24586776 -0.4054287  -0.07417434 -0.13182136
  0.06123444 -0.11323699 -0.38463914 -0.06395227 -0.415

In [15]:
#TF-IDF with scikit-learn:
def tfidf_similarity(text1):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(text1)
    return X, vectorizer

X, vectorizer = tfidf_similarity(df['Text'])
similarity_score = cosine_similarity(X, X)
print(similarity_score)

[[1.         0.01361175 0.01403186 ... 0.03403756 0.01904735 0.01310931]
 [0.01361175 1.         0.09153991 ... 0.08192573 0.03405815 0.01489347]
 [0.01403186 0.09153991 1.         ... 0.04428513 0.04458582 0.03509888]
 ...
 [0.03403756 0.08192573 0.04428513 ... 1.         0.03968907 0.01465114]
 [0.01904735 0.03405815 0.04458582 ... 0.03968907 1.         0.01856889]
 [0.01310931 0.01489347 0.03509888 ... 0.01465114 0.01856889 1.        ]]
